In [7]:
# Para usar o sleep:
import time
import re
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
# Contador
from itertools import count
# Manipulação de JSON:
import json
from dictdiffer import diff
# Para calcular similaridade entre strings:
from thefuzz import fuzz

Inicializando o logger:

In [8]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [9]:
def load_json(file_path: str) -> dict:
    """
    Carrega o JSON armazenado no caminho informado.
    Parâmetros
    ----------
    file_path
        Caminho para o arquivo JSON armazenado em disco.
    Retorno
    -------
    dict
        JSON na forma de um dicionário.
    """
    with open(file_path, "r", encoding="utf-8") as readfile:
        data = json.load(readfile)
    return data

def export_json(json_file: dict, file_path: str):
    """
    Exporta o dicionário como um JSON.
    Parâmetros
    ----------
    json_file
        JSON na forma de dicionário.
    file_path
        Caminho onde o JSON deverá ser armazenado em disco.
    Retorno
    -------
    """
    with open(file_path, "w", encoding="utf-8") as outfile:
        json.dump(json_file, outfile, indent = 4, ensure_ascii=False)

### Obtendo os usuários brasileiros a partir do arquivo brazilian_users.json:

In [10]:
path = '../data/'
users_file_name = "brazilian_users"

users_file = load_json(path + users_file_name + ".json")
number_of_users = len(users_file['brazilian_users'])

print(users_file)

{'brazilian_users': ['/user/isaiascsc', '/user/ismaelmpr', '/user/israelleoni', '/user/ivomartinez', '/user/j133y', '/user/jaadubya', '/user/jackson037', '/user/jakeline_ferreira', '/user/jamileagatha', '/user/jander68', '/user/jbortolaz', '/user/jckluska', '/user/jcleiton', '/user/jeffsbacks', '/user/jgavioli', '/user/jhenrique', '/user/jjessicaferreira', '/user/jjunior', '/user/jjunot', '/user/jmpadua', '/user/joao-vasques', '/user/joao_paulo_ribeiro', '/user/joaocym', '/user/joaogsanchez', '/user/joaomario', '/user/joaomeisen', '/user/joaopkg', '/user/joaoyodog', '/user/jojana', '/user/jonatanpaulini', '/user/jorgedoval', '/user/josuefernando', '/user/juanpablonunes', '/user/juarezjms', '/user/julianaram', '/user/junqneto', '/user/jvictor2907', '/user/jvitorsantos', '/user/kaiojag', '/user/kalumaurer', '/user/kanunes', '/user/kapo_did', '/user/kasku', '/user/kidrs', '/user/kitoandrade', '/user/kkjf02', '/user/kleecioss', '/user/koteca', '/user/kvasconcelos14', '/user/kyllercg', '/us

### Definindo credenciais para login e nomes dos arquivos

In [11]:
base_url = 'https://untappd.com'

dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

browser = webdriver.Chrome(options=options)

browser.get(base_url + '/login')
browser.find_element(By.NAME,"username").send_keys(EMAIL)
browser.find_element(By.NAME,"password").send_keys(PASSWORD)

time.sleep(20)


users_friends_export_first_level = users_file_name+"_first_level_friends.json"
users_friends_export_second_level = users_file_name+"_second_level_friends.json"

# Setar flags de acordo com qual nível de amigos está sendo processado 
users_friends_flag_first_level = True
users_friends_flag_second_level = False

user_data = {}
user_friends_first_level_json = ''
user_friends_second_level_json = ''

### Acessando página de amigos de cada usuário e armazenando informações:

In [12]:

try:
    logger.info("Iniciando uma sessão...")
    with requests.Session() as session:
        logger.info("A iteração começará agora...")
        iteration_time = Clock("Coleta de amigos dos usuários")
        
        with tqdm(total=number_of_users) as progress_bar:
            progress_bar.set_description("Coletando os IDs de amigos de usuários do arquivo " + users_file_name)
            wait = WebDriverWait(browser, 3)
            page_counter = count(1)

            # Iterando sobre usuários do arquivo em questão
            for user_url in users_file['brazilian_users']:
                browser.get(base_url + user_url)
                full_page = browser.page_source
                a_tag = SoupStrainer('a')
                soup = BeautifulSoup(full_page, 'html.parser', parse_only=a_tag)
                counter = count(1)
                
                # Clicando no botão com número de amigos na página principal de um usuário que redireciona para sua lista de amigos
                element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-href=':stats/friends']")))
                browser.execute_script("arguments[0].click();", element)
                time.sleep(1.7)
                logger.info(f"Link 'Amigos' da {str(page_counter)[6:-1]}ª página clicado para o usuário{user_url}")

                # Validando e clicando no Show More para capturar todos os amigos do usuário atual
                while True:
                    try:
                        element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[class='yellow button more more_friends']")))
                        browser.execute_script("arguments[0].click();", element)
                        time.sleep(1.7)
                        logger.info(f"Link 'Show more' da {str(page_counter)[6:-1]}ª página clicado pela {next(counter)}ª vez")
                    except:
                        logger.info('Fim da página alcançado!')
                        next(page_counter)
                        break
                
                # Buscando lista de amigos na página atual
                full_page = browser.page_source
                li_tag = SoupStrainer('li')
                soup = BeautifulSoup(full_page, 'html.parser', parse_only=li_tag)
                friends = soup.find_all("li", {"class": "friend-item"})

                current_user_friends = []

                # Capturando amigos do usuário atual
                for friend in friends:
                    friend_url = friend.div.a['href']
                    current_user_friends.append(friend_url)

                user_data[user_url] = current_user_friends
                # Exportando arquivo de acordo com nível de amigos
                if(users_friends_flag_first_level == True):
                    user_friends_first_level_json = json.dumps(user_data)
                    json_object = json.loads(user_friends_first_level_json)
                    export_json(json_object, path + users_friends_export_first_level)
                else:
                    user_friends_second_level_json = json.dumps(user_data)
                    json_object = json.loads(user_friends_second_level_json)
                    export_json(json_object, path + users_friends_export_second_level)

                # Finalizando captura na página atual
                logger.info(f"Foram coletados {len(current_user_friends)} amigos do usuário {user_url} da página completa.")
                progress_bar.update(1)

            # Exportando arquivo de acordo com nível de amigos
            if(users_friends_flag_first_level == True):
                user_friends_first_level_json = json.dumps(user_data)
                json_object = json.loads(user_friends_first_level_json)
                export_json(json_object, path + users_friends_export_first_level)
            else:
                user_friends_second_level_json = json.dumps(user_data)
                json_object = json.loads(user_friends_second_level_json)
                export_json(json_object, path + users_friends_export_second_level)
            
        iteration_time.stop_watch()
        logger.info("Coleta concluída!")

# Em caso de algum erro ao longo do processo, salva o que foi possível armazenar
except:
    # Exportando arquivo de acordo com nível de amigos
    if(users_friends_flag_first_level == True):
        user_friends_first_level_json = json.dumps(user_data)
        json_object = json.loads(user_friends_first_level_json)
        export_json(json_object, path + users_friends_export_first_level)
    else:
        user_friends_second_level_json = json.dumps(user_data)
        json_object = json.loads(user_friends_second_level_json)
        export_json(json_object, path + users_friends_export_second_level)


2023-09-25 09:00:14,673 - INFO - Iniciando uma sessão...
2023-09-25 09:00:14,673 - INFO - Iniciando uma sessão...
2023-09-25 09:00:14,676 - INFO - A iteração começará agora...
2023-09-25 09:00:14,676 - INFO - A iteração começará agora...
Coletando os IDs de amigos de usuários do arquivo test:   0%|          | 0/366 [00:00<?, ?it/s]2023-09-25 09:00:21,657 - INFO - Link 'Amigos' da 1ª página clicado para o usuário/user/isaiascsc
2023-09-25 09:00:21,657 - INFO - Link 'Amigos' da 1ª página clicado para o usuário/user/isaiascsc
2023-09-25 09:00:23,408 - INFO - Link 'Show more' da 1ª página clicado pela 1ª vez
2023-09-25 09:00:23,408 - INFO - Link 'Show more' da 1ª página clicado pela 1ª vez
2023-09-25 09:00:25,160 - INFO - Link 'Show more' da 1ª página clicado pela 2ª vez
2023-09-25 09:00:25,160 - INFO - Link 'Show more' da 1ª página clicado pela 2ª vez
2023-09-25 09:00:28,290 - INFO - Fim da página alcançado!
2023-09-25 09:00:28,290 - INFO - Fim da página alcançado!
2023-09-25 09:00:28,437